In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

In [ ]:
tf_idf_norm = pd.read_csv(PATH+"//Normalized TF_IDF for project type and target for each step of effective path.csv").set_index(["Step",'index'])



In [ ]:
# separating the living situation and the exit destination into categories
goal = [10,11]
closer = [3,19,20,21,22,23,25,26,28,31]
trans = [1,2,4,5,6,12,13,14,15,18,27,29]
no_progress = [7,16]
hard_to_judge = [8,9,17,24,30,99]

In [ ]:
# Functions used:

# recording effective pathway
def effective_pathway_with_exit(df,df1,target,target_label):
    client = df1.ClientID.unique()
    pathway = pd.DataFrame()
    for i in range(len(client)):
        temp = df1[df1['ClientID']==client[i]]
        temp1 = df[df['ClientID']==client[i]]
        for k in range(len(target)):
            if temp1['Destination'].iloc[-1] in target[k]:
                exit_type = target_label[k]
            if temp1['LivingSituation'].iloc[0] in target[k]:
                entry_type = target_label[k]
        pathway = pathway.append({"ClientID":client[i],"path":[entry_type]+list(temp['ProjectType'].unique())+[exit_type],"length": len(temp['ProjectType'].unique())},ignore_index=True)
    return pathway
    


In [ ]:
pathway = effective_pathway_with_exit(df,df1,[goal,closer,trans,no_progress,hard_to_judge],['ultimate goal','closer to exit','transitional phase','no progress or worse','hard to judge'])

In [ ]:
N = pathway.length.unique()
cosine = pd.DataFrame(columns = ['length','similarity','target'])
for n in N:
    temp = pathway[pathway['length']==n]
    for i in range(len(temp)):
        cos = pd.DataFrame(tf_idf_norm.loc[1].loc[str(temp.iloc[i]['path'][0])])    
        x = pd.DataFrame(tf_idf_norm.loc[n].loc['ultimate goal'])
        cos = cos.T.append(x.T,ignore_index=True)
        if temp.iloc[i]['path'][-1] == 'ultimate goal':
            target = 'ultimate goal'
        elif temp.iloc[i]['path'][-1] == 'closer to exit' or temp.iloc[i]['path'][-1] == 'transitional phase':
            target = 'closer or trans'
        elif temp.iloc[i]['path'][-1] == 'no progress or worse' or temp.iloc[i]['path'][-1] == 'hard to judge':
            target = 'unsuccessful'
        cosine = cosine.append({'length':n,'similarity':cosine_similarity(cos)[0][1],"target":target},ignore_index = True)            

In [ ]:
fontsize = 25
cosine_avg = cosine.groupby(['target','length']).mean()
plt.figure(figsize = (20,10))
x = -0.2
for i in ['ultimate goal','closer or trans','unsuccessful']:
    if i == 'ultimate goal':
        color = 'g'
    elif i == 'closer or trans':
        color = 'y'
    elif i == 'unsuccessful':
        color = 'r'
    plt.bar(cosine_avg.loc[i].index+x,cosine_avg.loc[i]['similarity'],color = color,width = 0.2,label=i)
    x = x+0.2
ax = plt.legend(fontsize=fontsize)
ax = plt.xticks([1,2,3,4],fontsize=fontsize)
ax = plt.yticks(fontsize=fontsize)

plt.savefig(PATH + "//Similarity of the initial living situation with ultimate goal.png")